# IT Incident SLA Comliance Analysis

Service Level Agreement (SLA) compliance represents a critical performance metric in IT service management, directly impacting customer satisfaction, operational efficiency, and business continuity. This analysis examines cleaned IT incident data to identify systematic patterns affecting SLA compliance rates and builds predictive models to forecast potential SLA breaches.

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
import warnings
import shap
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',
              None)  # Display all columns in DataFrame output.
pd.set_option('display.max_rows',
              None)  # Display all rows in DataFrame output.
# Load data from dataset
df = pd.read_csv('../data/incidents_cleaned.csv')

# Display DataFrame information
df.info()
print(f"\nDataset shape: {df.shape}")
print(f"\nTarget Variable Distribution:")
print(df['made_sla'].value_counts())
print(f"SLA Compliance Rate: {df['made_sla'].mean():.1%}")
print(f"SLA Breach Rate: {(1 - df['made_sla'].mean()):.1%}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6729 entries, 0 to 6728
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   number                   6729 non-null   object
 1   incident_state           6729 non-null   object
 2   active                   6729 non-null   bool  
 3   reassignment_count       6729 non-null   int64 
 4   reopen_count             6729 non-null   int64 
 5   sys_mod_count            6729 non-null   int64 
 6   made_sla                 6729 non-null   bool  
 7   caller_id                6727 non-null   object
 8   opened_by                6432 non-null   object
 9   opened_at                6729 non-null   object
 10  sys_created_by           3784 non-null   object
 11  sys_created_at           3784 non-null   object
 12  sys_updated_by           6729 non-null   object
 13  sys_updated_at           6729 non-null   object
 14  contact_type             6729 non-null  

## 1. Feature Engineering

### 1.1 Operational Complexity Features

In [37]:
df_fe = df.copy()

Rationale: EDA revealed sys_mod_count (r=-0.37) and reassignment_count (r=-0.25) as top predictors. Raw counts don't capture intensity (speed of handling), so we create ratio features normalized by incident duration.

In [38]:
# Raw operational counts (available during incident)
# sys_mod_count: System modifications made so far
# reassignment_count: Number of reassignments that occurred

# Weighted activity score (MODIFIED: exclude reopen_count)
# Weights: modifications (1x), reassignments (2x)
df_fe['activity_score'] = (df_fe['sys_mod_count'] +
                           df_fe['reassignment_count'] * 2)

# Binary complexity flags
df_fe['has_reassignment'] = (df_fe['reassignment_count'] > 0).astype(int)
df_fe['high_modification'] = (df_fe['sys_mod_count'] > 3).astype(int)

# Simplified complexity flag (MODIFIED: exclude reopen_count condition)
df_fe['is_complex'] = ((df_fe['sys_mod_count'] > 4) |
                       (df_fe['reassignment_count'] > 2)).astype(int)

print("✓ Created 3 operational complexity features:")
print("  - activity_score: mods + 2×reassignments (reopen excluded)")
print("  - has_reassignment, high_modification (binary flags)")
print("  - is_complex: >4 mods OR >2 reassignments")

# Validation
complex_sla = df_fe.groupby('is_complex')['made_sla'].mean()
print(f"\nValidation - is_complex feature:")
print(f"  Simple: {complex_sla[0]:.1%}, Complex: {complex_sla[1]:.1%}")
print(f"  Gap: {(complex_sla[0]-complex_sla[1])*100:.1f}%")

✓ Created 3 operational complexity features:
  - activity_score: mods + 2×reassignments (reopen excluded)
  - has_reassignment, high_modification (binary flags)
  - is_complex: >4 mods OR >2 reassignments

Validation - is_complex feature:
  Simple: 98.7%, Complex: 69.4%
  Gap: 29.2%


### 1.2 Severity Features

Rationale: Priority, Impact, and Urgency show strong multicollinearity (r=0.75-0.89), indicating they capture overlapping severity information. Using all three would introduce redundancy. We create binary "high severity" flags instead. EDA showed Priority has widest performance gap (Critical: 53.3% vs Low: 95.5% = 42.2% gap)From EDA: Priority shows 42.2% gap, but high multicollinearity with impact/urgency

In [39]:
# Binary severity flags (convert categorical to 0/1)
df_fe['is_high_priority'] = df_fe['priority'].isin(
    ['1 - Critical', '2 - High']).astype(int)
df_fe['is_high_impact'] = df_fe['impact'].isin(['1 - High']).astype(int)
df_fe['is_high_urgency'] = df_fe['urgency'].isin(['1 - High']).astype(int)
df_fe['is_high_severity'] = ((df_fe['is_high_priority'] == 1) |
                             (df_fe['is_high_impact'] == 1) |
                             (df_fe['is_high_urgency'] == 1)).astype(int)

print("✓ Created 4 severity-based features")

sev_sla = df_fe.groupby('is_high_severity')['made_sla'].mean()
print(
    f"Validation: Low {sev_sla[0]:.1%}, High {sev_sla[1]:.1%}, Gap {(sev_sla[0]-sev_sla[1])*100:.1f}%"
)

✓ Created 4 severity-based features
Validation: Low 87.4%, High 54.9%, Gap 32.5%


### 1.3 Interaction Features

Rationale: Combinations of factors may have compounding effects on SLA compliance. High severity + High complexity likely exhibits multiplicative negative impact rather than simple additive effect.

In [40]:
# Severity + Complexity
df_fe['severity_complexity'] = (df_fe['is_high_severity'] * df_fe['is_complex']).astype(int)

# Priority confirmation + Complexity  
# Note: u_priority_confirmation is available at incident time (procedural flag)
df_fe['confirmed_complex'] = (df_fe['u_priority_confirmation'] * df_fe['is_complex']).astype(int)

# High priority + Reassignment
df_fe['priority_reassign'] = (df_fe['is_high_priority'] * df_fe['has_reassignment']).astype(int)

print("✓ Created 3 interaction features")

risk_sla = df_fe.groupby('severity_complexity')['made_sla'].mean()
print(f"Validation: No risk {risk_sla[0]:.1%}, High risk {risk_sla[1]:.1%}, Gap {(risk_sla[0]-risk_sla[1])*100:.1f}%")

print("\n" + "-"*80)
print(f"Feature Engineering Summary:")
print(f"  Original: {df.shape[1]} → Engineered: {df_fe.shape[1]} (+{df_fe.shape[1]-df.shape[1]} features)")
print(f"  Excluded post-incident metrics: closed_time, reopen_count, per_day ratios")
print("-"*80)

✓ Created 3 interaction features
Validation: No risk 87.3%, High risk 50.3%, Gap 36.9%

--------------------------------------------------------------------------------
Feature Engineering Summary:
  Original: 32 → Engineered: 43 (+11 features)
  Excluded post-incident metrics: closed_time, reopen_count, per_day ratios
--------------------------------------------------------------------------------


### 1.4 Feature Encoding

#### 1.4.1 Ordinal Encoding

Priority, Impact, and Urgency have natural ordering from low to high severity. Preserve this ordering with numeric encoding

In [41]:
df_enc = df_fe.copy()

In [42]:
# Priority (1=Low, 4=Critical)
priority_map = {
    '4 - Low': 1,
    '3 - Moderate': 2,
    '2 - High': 3,
    '1 - Critical': 4
}
df_enc['priority_enc'] = df_enc['priority'].map(priority_map)

# Impact (1=Low, 3=High)
impact_map = {'3 - Low': 1, '2 - Medium': 2, '1 - High': 3}
df_enc['impact_enc'] = df_enc['impact'].map(impact_map)

# Urgency (1=Low, 3=High)
urgency_map = {'3 - Low': 1, '2 - Medium': 2, '1 - High': 3}
df_enc['urgency_enc'] = df_enc['urgency'].map(urgency_map)

print("✓ Ordinal encoded 3 variables:")
print("  - priority_enc: 1 (Low) → 4 (Critical)")
print("  - impact_enc: 1 (Low) → 3 (High)")
print("  - urgency_enc: 1 (Low) → 3 (High)")


✓ Ordinal encoded 3 variables:
  - priority_enc: 1 (Low) → 4 (Critical)
  - impact_enc: 1 (Low) → 3 (High)
  - urgency_enc: 1 (Low) → 3 (High)


#### 1.4.2 Label Encoding

assignment_group and category have many unique values without natural ordering

In [43]:
# assignment_group (from EDA: 56% performance variance across groups)
le_group = LabelEncoder()
df_enc['assignment_group'] = df_enc['assignment_group'].fillna('Unknown')
df_enc['assignment_group_enc'] = le_group.fit_transform(
    df_enc['assignment_group'])

# category (from EDA: 21.1% performance gap)
le_cat = LabelEncoder()
df_enc['category'] = df_enc['category'].fillna('Unknown')
df_enc['category_enc'] = le_cat.fit_transform(df_enc['category'])

print(f"✓ Label encoded 2 variables:")
print(f"  - assignment_group_enc: {len(le_group.classes_)} unique groups")
print(f"  - category_enc: {len(le_cat.classes_)} unique categories")

✓ Label encoded 2 variables:
  - assignment_group_enc: 65 unique groups
  - category_enc: 47 unique categories


#### 1.4.3 Binary Encoding

Binary features created during feature engineering need to be explicitly typed as integers

In [44]:
binary_vars = [
    'knowledge', 'u_priority_confirmation', 'made_sla', 'has_reassignment',
    'high_modification', 'is_complex', 'is_high_priority', 'is_high_impact',
    'is_high_urgency', 'is_high_severity', 'severity_complexity',
    'confirmed_complex', 'priority_reassign'
]

for var in binary_vars:
    if var in df_enc.columns:
        df_enc[var] = df_enc[var].astype(int)

verified_count = len([v for v in binary_vars if v in df_enc.columns])
print(f"✓ Verified {verified_count} binary variables as integer type")

✓ Verified 13 binary variables as integer type


### 2. Feature Selection

Feature selection aims to identify the most predictive features while reducing dimensionality and preventing overfitting. We employ two complementary methods:
1. Filter Method (ANOVA F-test):
   - Statistical test measuring relationship between each feature and target
   - Fast, model-agnostic approach
   - Identifies univariate relationships
2. Embedded Method (Random Forest Feature Importance):
   - Features selected during model training
   - Captures feature interactions and non-linear relationships
   - Specific to tree-based models
We combine both methods using the union of selected features to ensure
we capture both statistical significance and model-specific importance.

### 2.1 Define Candidate Features

Prepare feature groups based on their types and encoding methods

In [50]:
# Define target variable
target = 'made_sla'

# Numerical features (continuous and ordinal encoded)
numerical_features = [
    'priority_enc',
    'impact_enc',
    'urgency_enc',  # Ordinal encoded severity
    'reassignment_count',
    'sys_mod_count',  # Raw operational counts
    'activity_score'  # Weighted complexity
]

# Categorical features (label encoded)
categorical_features = [
    'assignment_group_enc',  # Support team assignment
    'category_enc'  # Incident category
]

# Binary features (0/1 flags)
binary_features = [
    'knowledge',
    'u_priority_confirmation',  # Process flags
    'has_reassignment',
    'high_modification',
    'is_complex',  # Complexity flags
    'is_high_priority',
    'is_high_impact',
    'is_high_urgency',
    'is_high_severity',  # Severity flags
    'severity_complexity',
    'confirmed_complex',
    'priority_reassign'  # Interaction flags
]

# Combine all candidate features
all_features = numerical_features + categorical_features + binary_features
available_features = [f for f in all_features if f in df_enc.columns]

print(f"Candidate Features Summary:")
print(f"  Total: {len(available_features)} features")
print(
    f"    - Numerical (including ordinal): {len([f for f in numerical_features if f in df_enc.columns])}"
)
print(
    f"    - Categorical (label encoded): {len([f for f in categorical_features if f in df_enc.columns])}"
)
print(
    f"    - Binary (flags): {len([f for f in binary_features if f in df_enc.columns])}"
)

X = df_enc[available_features].copy()
y = df_enc[target].copy()
X = X.fillna(X.median())

print(f"\nX shape: {X.shape}, y shape: {y.shape}")

Candidate Features Summary:
  Total: 20 features
    - Numerical (including ordinal): 6
    - Categorical (label encoded): 2
    - Binary (flags): 12

X shape: (6729, 20), y shape: (6729,)


### 2.2 Prepare Feature Matrix and Target Vector

Extract features and target, handle missing values

In [51]:
# Prepare X (features) and y (target)
X = df_enc[available_features].copy()
y = df_enc[target].copy()

# Handle missing values using median imputation
# Median is robust to outliers and appropriate for numerical/ordinal features
X = X.fillna(X.median())

print(f"Feature matrix (X) shape: {X.shape}")
print(f"Target vector (y) shape: {y.shape}")
print(f"\nTarget distribution:")
print(f"  Class 1 (SLA Met): {y.sum():,} ({y.mean():.1%})")
print(
    f"  Class 0 (SLA Breach): {(~y.astype(bool)).sum():,} ({(1-y.mean()):.1%})"
)
print(f"  Class imbalance ratio: {y.sum() / (~y.astype(bool)).sum():.2f}:1")


Feature matrix (X) shape: (6729, 20)
Target vector (y) shape: (6729,)

Target distribution:
  Class 1 (SLA Met): 5,813 (86.4%)
  Class 0 (SLA Breach): 916 (13.6%)
  Class imbalance ratio: 6.35:1


### 2.3 Filter Method: ANOVA F-test

This code selects the k best features from X that have the strongest statistical relationship with y according to the ANOVA F-test

In [53]:
k_best = 20
selector_anova = SelectKBest(score_func=f_classif, k=k_best)
X_anova = selector_anova.fit_transform(X, y)

anova_mask = selector_anova.get_support()
anova_features = X.columns[anova_mask].tolist()
anova_scores = selector_anova.scores_[anova_mask]

print(f"\nANOVA Top {k_best}:")
for rank, (feat, score) in enumerate(
        sorted(zip(anova_features, anova_scores),
               key=lambda x: x[1],
               reverse=True)[:10], 1):
    print(f"  {rank}. {feat:30s} F={score:.2f}")


ANOVA Top 20:
  1. confirmed_complex              F=1694.81
  2. is_complex                     F=1446.74
  3. activity_score                 F=1157.19
  4. high_modification              F=1095.61
  5. sys_mod_count                  F=1084.96
  6. reassignment_count             F=463.56
  7. has_reassignment               F=320.08
  8. is_high_priority               F=192.76
  9. is_high_severity               F=192.76
  10. u_priority_confirmation        F=190.19


### 2.4 Random Forest Feature Importance

In [55]:
rf = RandomForestClassifier(n_estimators=100,
                            max_depth=10,
                            random_state=42,
                            n_jobs=-1,
                            class_weight='balanced')
rf.fit(X, y)

importances = pd.Series(rf.feature_importances_,
                        index=X.columns).sort_values(ascending=False)
rf_features = importances.head(k_best).index.tolist()

print(f"\nRandom Forest Top {k_best}:")
for rank, (feat, imp) in enumerate(importances.head(10).items(), 1):
    print(f"  {rank}. {feat:30s} Imp={imp:.4f}")

selected_features = list(set(anova_features + rf_features))
selected_features.sort()

print(f"\nFinal: {len(selected_features)} features selected")

X_selected = X[selected_features].copy()


Random Forest Top 20:
  1. sys_mod_count                  Imp=0.2025
  2. activity_score                 Imp=0.1892
  3. confirmed_complex              Imp=0.1610
  4. is_complex                     Imp=0.1087
  5. high_modification              Imp=0.1003
  6. assignment_group_enc           Imp=0.0511
  7. category_enc                   Imp=0.0471
  8. reassignment_count             Imp=0.0258
  9. u_priority_confirmation        Imp=0.0238
  10. has_reassignment               Imp=0.0164

Final: 20 features selected


### 2.5 Combine Methods

In [57]:
selected_features = list(set(anova_features + rf_features))
selected_features.sort()

print(f"Results:")
print(f"  ANOVA: {len(anova_features)} features")
print(f"  Random Forest: {len(rf_features)} features")
print(f"  Union: {len(selected_features)} features")
print(f"  Overlap: {len(set(anova_features) & set(rf_features))} features")

print(f"\nFinal Selected Features ({len(selected_features)}):\n")
print(f"{'#':<4} {'Feature':<35} {'ANOVA':<8} {'RF':<8}")
print("-" * 57)
for i, feat in enumerate(selected_features, 1):
    in_anova = "✓" if feat in anova_features else " "
    in_rf = "✓" if feat in rf_features else " "
    print(f"{i:<4} {feat:<35} {in_anova:<8} {in_rf:<8}")

X_selected = X[selected_features].copy()

print(f"\nDimensionality Reduction:")
print(
    f"  Original: {X.shape[1]} → Selected: {X_selected.shape[1]} ({(1-X_selected.shape[1]/X.shape[1])*100:.1f}% reduction)"
)

Results:
  ANOVA: 20 features
  Random Forest: 20 features
  Union: 20 features
  Overlap: 20 features

Final Selected Features (20):

#    Feature                             ANOVA    RF      
---------------------------------------------------------
1    activity_score                      ✓        ✓       
2    assignment_group_enc                ✓        ✓       
3    category_enc                        ✓        ✓       
4    confirmed_complex                   ✓        ✓       
5    has_reassignment                    ✓        ✓       
6    high_modification                   ✓        ✓       
7    impact_enc                          ✓        ✓       
8    is_complex                          ✓        ✓       
9    is_high_impact                      ✓        ✓       
10   is_high_priority                    ✓        ✓       
11   is_high_severity                    ✓        ✓       
12   is_high_urgency                     ✓        ✓       
13   knowledge                          

### 3. Feature Scaling

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
X_scaled_df = pd.DataFrame(X_scaled,
                           columns=X_selected.columns,
                           index=X_selected.index)

print(f"\n✓ Scaled {X_scaled_df.shape[1]} features")

## 4. SMOTE
smote = SMOTE(random_state=42, k_neighbors=5)
X_resampled, y_resampled = smote.fit_resample(X_scaled_df, y)

print(f"\n✓ SMOTE: {len(y):,} → {len(y_resampled):,} samples")

## 5. Save
balanced_data = pd.DataFrame(X_resampled, columns=X_scaled_df.columns)
balanced_data['made_sla'] = y_resampled
balanced_data.to_csv('../data/incidents_smote_balanced.csv', index=False)

print(f"\n✓ Saved to 'incidents_smote_balanced.csv'")
print(f"  Shape: {balanced_data.shape}")
print(f"  Features: Pre-incident only (no post-incident leakage)")


✓ Scaled 20 features

✓ SMOTE: 6,729 → 11,626 samples

✓ Saved to 'incidents_smote_balanced.csv'
  Shape: (11626, 21)
  Features: Pre-incident only (no post-incident leakage)
